# Tools

This notebook contains all the tools that will be used by the agent.

In [ ]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [ ]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master


In [ ]:
# TODO: Explore the loaded dataframe


In [ ]:
# TODO: Add tool description


def get_latlng(city: str) -> any:
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }

In [ ]:
# TODO: Test get_latlng method
# case insensitive search


### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [ ]:
# TODO: Add tool description

def get_temperature(latitude: float, longitude: float) -> any:
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [ ]:
# TODO: Test get_temperature method


### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [ ]:
# TODO: Add tool description

def query_album_track(query: str) -> str:
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [ ]:
# TODO: Test the query_album_track function


### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [ ]:
class SQLiteTool(Tool):

   def __init__(self, db_file):
      self.db_file = db_file 
      self.setup()

   def setup(self):
      super().setup()
      self.conn = sqlite3.connect(self.db_file)

   def forward(self, query: str) -> str:
      try:
         cursor = self.conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      except Exception as e:
         print(f'Query exception: {e}')  